In [26]:
import pyscipopt
from pyscipopt import Model, quicksum, Expr
# Page 351 column generation method practice with SCIP

# build a toy example 
def build_RMP(x_cnt, col):
    if len(col)>0:        
        row_1.append(col[0])
        row_2.append(col[1])
        row_3.append(col[-1])
        x_cnt = x_cnt + 1
    
    my_toy = Model("RMP")    
    cons = []  
    x = []
    # ===========================================================================
    # Original problem
    for i in range(x_cnt):
        x.append(my_toy.addVar(name="x_"+str(i), vtype='C', lb=0, ub=None))
    
    # 约束1 小于等于 supply >> 首次迭代 >> 所有的 y(i)=1.0
    cons.append(my_toy.addCons(quicksum(row_1[i] * x[i] for i in range(x_cnt)) >= 46))
    cons.append(my_toy.addCons(quicksum(row_2[i] * x[i] for i in range(x_cnt)) >= 22))
    cons.append(my_toy.addCons(quicksum(row_3[i] * x[i] for i in range(x_cnt)) >= 43))

    # objective func                                                                      
    my_toy.setObjective(quicksum(x[i] for i in range(x_cnt)), sense="minimize")

    # ===========================================================================
    my_toy.hideOutput()

    my_toy.optimize()
    # my_toy.writeProblem('RMP.lp')
    print("RMP Model status:", my_toy.getStatus())
    y = []
    for i in cons:
        y.append(my_toy.getDualsolLinear(i))
        # y.append(my_toy.getRowDualSol(my_toy.getRowLinear(i)))
        
        # print("Dual multiplier getDualsolLinear()", my_toy.getDualsolLinear(i))
    # for i in cons:
    #     print("Dual multiplier getDualMultiplier()", my_toy.getDualMultiplier(i))
    # for i in cons:
    #     print("Dual multiplier getDualSolVal()", my_toy.getDualSolVal(i))
    # for i in cons:
    #     print("Dual multiplier getDualfarkasLinear()", my_toy.getDualfarkasLinear(i))
    # print("===========================================================================")    
    # for i in range(x_cnt):
    #     print("x(%i)  " % (i) + str(my_toy.getVal(x[i])))        
    # print("Solution:")
    # print("===========================================================================")
    return y, x_cnt

# build a toy example 
def build_SP(y):
    my_toy = Model("SP")
    a_cnt = 3
    alpha = []
    
    cons = []
    # ===========================================================================
    # Original problem
    for i in range(a_cnt):
        alpha.append(my_toy.addVar(name="a_"+str(i), vtype='I', lb=0, ub=None))
    
    # 约束1 小于等于 supply >> 首次迭代 >> 所有的 y(i)=1.0
    
    cons.append(my_toy.addCons(10.* alpha[0] + 11.* alpha[1] + 19.*alpha[2] <= 80))
    
    # objective func                                                                      
    my_toy.setObjective(1 - (y[0]*alpha[0] + y[1]*alpha[1] + y[2]*alpha[2]), sense="minimize")

    # ===========================================================================
    my_toy.hideOutput()
    my_toy.optimize()
    # my_toy.writeProblem('cg.lp')
    print("SP Model status:", my_toy.getStatus())
    # print("===========================================================================")
    column = []
    for i in range(a_cnt):
        column.append(my_toy.getVal(alpha[i]))
        # print("alpha(%i)  " % (i) + str(my_toy.getVal(alpha[i])))        
    
    # getSols: Retrieve list of all feasible primal solutions stored in the solution storage.
    # print(my_toy.getSols())
    obj = my_toy.getObjVal()
    print("SP objective value is: ", my_toy.getObjVal())
    if obj < 0:        
        print("New column:", column) 
        return column, obj
    else:
        return "No Negative Reduced-cost - no new column needed", 9999

    
# build a toy example 
def build_final(x_cnt):    
    my_toy = Model("Final")    
    cons = []  
    x = []
    # ===========================================================================
    # Original problem
    for i in range(x_cnt):
        x.append(my_toy.addVar(name="x_"+str(i), vtype='I', lb=0, ub=None))
    
    # 约束1 小于等于 supply >> 首次迭代 >> 所有的 y(i)=1.0
    cons.append(my_toy.addCons(quicksum(row_1[i] * x[i] for i in range(x_cnt)) >= 46))
    cons.append(my_toy.addCons(quicksum(row_2[i] * x[i] for i in range(x_cnt)) >= 22))
    cons.append(my_toy.addCons(quicksum(row_3[i] * x[i] for i in range(x_cnt)) >= 43))

    # objective func                                                                      
    my_toy.setObjective(quicksum(x[i] for i in range(x_cnt)), sense="minimize")

    # ===========================================================================
    my_toy.hideOutput()

    my_toy.optimize()
    
    print("RMP Model status:", my_toy.getStatus())
    # for i in cons:
    #     print("Dual multiplier getDualMultiplier()", my_toy.getDualMultiplier(i))
    # for i in cons:
    #     print("Dual multiplier getDualSolVal()", my_toy.getDualSolVal(i))
    # for i in cons:
    #     print("Dual multiplier getDualfarkasLinear()", my_toy.getDualfarkasLinear(i))
    # print("===========================================================================")    
    print(my_toy.getObjVal())
    for i in range(x_cnt):
        print("x(%i)  " % (i) + str(my_toy.getVal(x[i])))        
    # print("Solution:")
    # print("===========================================================================")
    
    
x_cnt = 5
row_1 = [8., 6., 6., 5., 5.]
row_2 = [0., 1., 0., 1., 2.]
row_3 = [0., 0., 1., 1., 0.]
col = []
obj = -9999
for i in range(6):
    if obj > 0:
        build_final(x_cnt)
        break
    print("=========================================================================== ")
    y, x_cnt = build_RMP(x_cnt, col) 
    print("Dual solution", y)
    print("")
    col, obj = build_SP(y)
    print("=========================================================================== ")
    
    

RMP Model status: optimal
Dual solution [-0.0, -0.0, 1.0]

SP Model status: optimal
SP objective value is:  -3.0
New column: [-0.0, -0.0, 4.0]
RMP Model status: optimal
Dual solution [-0.0, 0.5, 0.25]

SP Model status: optimal
SP objective value is:  -2.5
New column: [-0.0, 7.0, -0.0]
RMP Model status: optimal
Dual solution [0.12142857142857144, 0.14285714285714285, 0.25]

SP Model status: optimal
SP objective value is:  -0.03571428571428559
New column: [0.0, 2.0, 3.0]
RMP Model status: optimal
Dual solution [0.125, 0.125, 0.25]

SP Model status: optimal
SP objective value is:  0.0
RMP Model status: optimal
20.0
x(0)  6.0
x(1)  0.0
x(2)  0.0
x(3)  0.0
x(4)  0.0
x(5)  3.0
x(6)  0.0
x(7)  11.0


Model status: optimal
y= [-0.0, -0.0, 1.0]
Solution: [-0.0, -0.0, 4.0]
feasible solution found by trivial heuristic after 0.0 seconds, objective value 1.000000e+00
presolving:
(round 1, fast)       2 del vars, 1 del conss, 0 add conss, 1 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 3 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 1 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : -3.00000000000000e+00 (2 solutions)
Dual Bound         : -3.00000000000000e+00
Gap                : 0.00 %
